In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report


In [5]:
df = pd.read_csv('EN_combined_train_with_labels.csv.gz')

In [6]:
df

,id,ED,HWP,LQ80,PPR,SC,current_skewness,spectral_centroid_power,tail_charge_diff,current_kurtosis,total_power,time_to_main_peak,time_to_peak,late_over_early,energy_label,psd_label_low_avse,psd_label_high_avse,psd_label_dcr,psd_label_lq,tp0
0,0_train_0,3409.0,2299.0,-717094.898532,0.719376,0.034655,1.780811,107.276207,-0.598625,2.117825,1.709302e+09,85,85,0.987910,582.364295,False,True,True,True,957
1,1_train_0,3404.0,2446.0,-331957.541919,0.729709,0.035314,1.756635,108.213621,-0.573984,2.058622,2.991376e+08,87,87,0.988301,250.159995,False,True,True,True,948
2,2_train_0,3411.0,2262.0,-425532.152706,0.715390,0.034915,1.079789,105.735183,-0.620023,-0.067003,6.244385e+08,95,95,0.987491,1212.323954,False,True,False,True,965
3,3_train_0,3408.0,2833.0,-306980.459766,0.769375,0.034752,1.899438,107.946935,-0.571186,2.443885,2.811583e+08,116,116,0.988450,240.878110,False,True,True,False,927
4,4_train_0,3406.0,2397.0,-362746.925366,0.728165,0.035132,1.156442,106.350372,-0.563259,-0.173890,3.885442e+08,94,94,0.988541,285.124189,False,True,True,False,958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039995,1039995_train_15,3403.0,2533.0,-278607.352936,0.737282,0.034943,1.997380,107.786931,-0.702128,2.026649,2.133856e+08,90,90,0.985736,210.834626,True,True,True,True,949
1039996,1039996_train_15,3404.0,2426.0,-477777.966558,0.726646,0.034483,1.814844,107.271038,-0.600863,1.943512,6.842378e+08,82,82,0.987814,380.170340,True,True,True,True,949
1039997,1039997_train_15,3409.0,2417.0,-466589.983952,0.730722,0.034517,1.694094,107.943127,-0.600439,1.601361,6.641180e+08,102,102,0.987833,370.109563,False,True,True,True,947
1039998,1039998_train_15,3400.0,3799.0,-143212.214717,0.776054,0.036649,2.031583,107.577145,-0.491736,3.825219,4.630840e+07,92,92,0.989948,98.258524,True,True,True,False,921


# MODEL TRAINING

## Eunice Model Training - psd_label_lq

### Baseline Model

## Nomin Model Training - psd_label_high_avse

### Baseline Model

## Prithvi Model Training - psd_label_low_avse

### Baseline Model

## Jade Model Training - psd_label_dcr

### Baseline Model